In [3]:
pip install pyodbc

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import pyodbc
import datetime
from datetime import datetime


In [ ]:
import datetime
from datetime import datetime
import pyodbc 
conn = pyodbc.connect(                       
    'Driver={SQL Server};'                   
    'Server=DESKTOP-25TJ8LJ\\SQLEXPRESS;'    
    'Database=menudb;'                       
    'Trusted_Connection=yes;'
)

cursor = conn.cursor()                                     
cursor.execute("SELECT ItemID, ItemName, Price from Menu")       
rows=cursor.fetchall()                                     
print("\n"+"-"*40)
print("\n            MENU ")
print("-"*40)
print("\n{:<5} {:<20} {:>10}".format("ID","Item  Name","Price"))
print("-"*40)
for row in rows:
    print("{:<5} {:<20} {:>10}".format(row[0],row[1],row[2])) 
choice=input("\nDo you want to place order?(yes/no): ").strip().lower()
if choice != "yes":
    print("Thank you!")
    conn.close()
else:
    print("Enter  Customer Details")
    name=input("Enter name: ")
    mobile=input("Enter mobile number: ")
    total_amount=0   #store final bill
    order_items=[]   #stores each ordered item temporarily
    while  True:
        item_name=input("Enter Item name(or type 'done' to finish):")
        if  item_name.lower()=="done":
            break
        quantity =int(input("Enter Quantity: "))
        cursor.execute("SELECT ItemId,price from Menu where lower(ItemName)=lower(?)",(item_name,))
        result=cursor.fetchone()
        if result:
            item_id=result[0]
            price=result[1]
            amount=price*quantity
            total_amount += amount
            order_items.append({"id":item_id,
                                "name":item_name,
                                "quantity":quantity,
                                "amount":amount}) 
        else:
            print("Item not found!")
    print("\n"+"="*45)
    print("Hotel Bill".center(45))
    print("="*45)
    print(f"\nCustomer Name:{name}")
    print(f"Mobile       :{mobile}")
    print(f"Date & Time  :{datetime.now().strftime('%d-%m-%Y %H:%M:%S')}")
    print("-"*45)

    print("{:<20} {:>10} {:>10}".format("Item","Qty","Amount"))
    print("-"*45)
    for item in order_items:
        print("{:<20} {:>10} {:>10.2f}".format(
            item['name'],
            item['quantity'],
            item['amount']))
    print("-"*45)
    print("{:<20} {:>8} {:>12.2f}".format("","Total:",total_amount))
    print("="*45)
    cursor.execute("Insert Into Customers (Name,mobile) Values (?, ?)",
    (name,mobile))
    conn.commit
    cursor.execute("Select @@IDENTITY")
    customer_id=cursor.fetchone()[0]
    for item in order_items:
        cursor.execute("""Insert into Orders
                       (CustomerID,ItemID, Quantity,  Amount,OrderDate)
                       values(?,?,?,?,Getdate())
                       """,
                       (
                       customer_id,
                       item['id'],
                       item['quantity'],
                       item['amount']
                       )
                    )
        conn.commit()
   
    conn.close()


----------------------------------------

            MENU 
----------------------------------------

ID    Item  Name                Price
----------------------------------------
1     Vada Pav                  20.00
2     Upma                      30.00
3     Samosa                    15.00
4     Sandwich                  40.00
5     Pav Bhaji                 80.00
6     Masala Dosa               70.00
7     Plain Dosa                50.00
8     Cold Coffee               60.00
9     Lassi                     45.00
10    Pizza                    120.00
11    Burger                    90.00
12    French Fries              70.00
13    Paneer Tikka             150.00
14    Biryani                  180.00
15    Thali                    200.00


Enter  Customer Details

                  Hotel Bill                 

Customer Name:Aarti
Mobile       :6374683746
Date & Time  :16-02-2026 15:28:57
---------------------------------------------
Item                        Qty     Amount
---------------------------------------------
pizza                         2     240.00
cold coffee                   3     180.00
---------------------------------------------
                       Total:       420.00
